# chfem Python API Tutorial

In [ ]:
# install chfem
!pip install git+https://gitlab.com/cortezpedro/chfem_gpu.git@dev -q

# To enable c stdout printing in colab notebook
!pip install wurlitzer -q
import wurlitzer
wurlitzer.Wurlitzer.flush_interval=0.001
%load_ext wurlitzer

# necessary for pyvista plots
!pip install pyvista -q
!pip install piglet -q
!pip install pyvirtualdisplay -q
!apt-get -qq install xvfb
from pyvirtualdisplay import Display
display = Display(visible=0, size=(600, 400))
display.start()
def pv_plot(array):
  grid = pv.ImageData()
  grid.dimensions = array.shape
  grid.point_data["scalars"] = array.flatten(order="F")
  plotter = pv.Plotter(notebook=True)
  plotter.add_volume(grid, scalars="scalars", cmap="jet")
  plotter.show()

# other imports
from matplotlib import pyplot as plt
import pyvista as pv
import numpy as np
import chfem

In [ ]:
def create_cube_array(array_side, cube_side, ntiles):
    cube = np.zeros((array_side, array_side, array_side), dtype=np.uint8)
    start_cube = array_side // 4
    end_cube = start_cube + cube_side
    cube[start_cube:end_cube, start_cube:end_cube, start_cube:end_cube] = 1
    cube_array = np.tile(cube, (ntiles, ntiles, ntiles))
    return cube_array

left = create_cube_array(10, 5, 10)
right = create_cube_array(20, 10, 5)
array = np.concatenate((left, right), axis=0)
array[array == 1] = 255

pv_plot(array)

In [ ]:
keff = chfem.compute_property('conductivity', array, mat_props={255: 1, 0: 0.1}, direction='x', output_fields="cubes")

temperature = chfem.import_scalar_field_from_chfem("cubes_temperature_0.bin", array.shape, rotate_domain=True)
pv_plot(temperature)

In [ ]:
Ceff = chfem.compute_property('elasticity', array, mat_props={255: (200, 0.2), 0: (100, 0.3)}, direction='x', output_fields="cubes")

displacement = chfem.import_vector_field_from_chfem("cubes_displacement_0.bin", array.shape, rotate_domain=True)
pv_plot(displacement[:, :, :, 0])

In [ ]:
Keff = chfem.compute_property('permeability', array, direction='x', output_fields="cubes")

pressure = chfem.import_scalar_field_from_chfem("cubes_pressure_0.bin", array.shape, rotate_domain=True)
pv_plot(pressure)